<a href="https://colab.research.google.com/github/DrigoDomingos/onfarm/blob/master/onfarm_dl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Upload the files

In [8]:
!unzip dp_model.zip

Archive:  dp_model.zip
   creating: dp_model/
  inflating: dp_model/.gitignore     
   creating: dp_model/Etapa1/
   creating: dp_model/Etapa1/AzulClaro/
  inflating: dp_model/Etapa1/AzulClaro/1.jpg  
  inflating: dp_model/Etapa1/AzulClaro/1.xml  
  inflating: dp_model/Etapa1/AzulClaro/2.jpg  
  inflating: dp_model/Etapa1/AzulClaro/2.xml  
  inflating: dp_model/Etapa1/AzulClaro/3.jpg  
  inflating: dp_model/Etapa1/AzulClaro/3.xml  
  inflating: dp_model/Etapa1/AzulClaro/4.jpg  
  inflating: dp_model/Etapa1/AzulClaro/4.xml  
  inflating: dp_model/Etapa1/AzulClaro/5.jpg  
  inflating: dp_model/Etapa1/AzulClaro/5.xml  
   creating: dp_model/keras_frcnn/
  inflating: dp_model/keras_frcnn/config.py  
  inflating: dp_model/keras_frcnn/data_augment.py  
  inflating: dp_model/keras_frcnn/data_generators.py  
  inflating: dp_model/keras_frcnn/FixedBatchNormalization.py  
  inflating: dp_model/keras_frcnn/losses.py  
  inflating: dp_model/keras_frcnn/pascal_voc_parser.py  
  inflating: dp_model/

In [9]:
cd dp_model/

/content/dp_model


# Convert XML to CSV

In [0]:
import os, sys, random
import xml.etree.ElementTree as ET
from glob import glob
import pandas as pd
from shutil import copyfile


def convert_xml2csv(path):
  annotations = glob(path)
  df = []
  cnt = 0
  for file in annotations:
      #filename = file.split('/')[-1].split('.')[0] + '.jpg'
      #filename = str(cnt) + '.jpg'
      filename = file.split('\\')[-1]
      filename =filename.split('.')[0] + '.jpg'
      row = []
      parsedXML = ET.parse(file)
      for node in parsedXML.getroot().iter('object'):
          blood_cells = node.find('name').text
          xmin = int(node.find('bndbox/xmin').text)
          xmax = int(node.find('bndbox/xmax').text)
          ymin = int(node.find('bndbox/ymin').text)
          ymax = int(node.find('bndbox/ymax').text)

          row = [filename, blood_cells, xmin, xmax, ymin, ymax]
          df.append(row)
          cnt += 1

  data = pd.DataFrame(df, columns=['filename', 'cell_type', 'xmin', 'xmax', 'ymin', 'ymax'])

  data[['filename', 'cell_type', 'xmin', 'xmax', 'ymin', 'ymax']].to_csv('test.csv', index=False)


#Azul claro
convert_xml2csv('/content/Etapa1/AzulClaro/*.xml')


# Install libraries

In [2]:
!pip install -r requirement.txt

In [16]:
cd ..

/content/dp_model


In [19]:
!python train_frcnn.py -o simple -p annotate.txt

Using TensorFlow backend.
Traceback (most recent call last):
  File "train_frcnn.py", line 79, in <module>
    all_imgs, classes_count, class_mapping = get_data(options.train_path)
  File "/content/dp_model/keras_frcnn/simple_parser.py", line 14, in get_data
    with open(input_path,'r') as f:
FileNotFoundError: [Errno 2] No such file or directory: 'annotate.txt'
